In [1]:
from chi import server, context
import chi, os, time, datetime

context.version = "1.0" 
context.choose_project()
context.choose_site(default="KVM@TACC")

In [2]:
username = os.getenv('USER') 
s = server.Server(
    f"node-persist-{username}", 
    image_name="CC-Ubuntu24.04",
    flavor_name="m1.large"
)
s.submit(idempotent=True)

Waiting for server node-persist-ahmed_offsechq_com's status to become ACTIVE. This typically takes 10 minutes for baremetal, but can take up to 20 minutes.


Server has moved to status ACTIVE


Attribute,node-persist-ahmed_offsechq_com
Id,409303e0-830a-4204-b5fb-615f79cfe562
Status,ACTIVE
Image Name,CC-Ubuntu24.04
Flavor Name,m1.large
Addresses,sharednet1: IP: 10.56.0.144 (v4) Type: fixed MAC: fa:16:3e:65:41:23
Network Name,sharednet1
Created At,2025-06-26T13:26:31Z
Keypair,ahmed_offsechq_com-jupyter
Reservation Id,None
Host Id,f873e03657a264d6bef7c4cf1f8a1d921944344e7f10ab418f462bbc


In [13]:
s

<Server 'node-persist-ahmed_offsechq_com'>

In [3]:
s.associate_floating_ip()

'129.114.27.235'

In [4]:
s.refresh()
s.show(type="widget")

Attribute,node-persist-ahmed_offsechq_com
Id,409303e0-830a-4204-b5fb-615f79cfe562
Status,ACTIVE
Image Name,CC-Ubuntu24.04
Flavor Name,m1.large
Addresses,sharednet1: IP: 10.56.0.144 (v4) Type: fixed MAC: fa:16:3e:65:41:23 IP: 129.114.27.235 (v4) Type: floating MAC: fa:16:3e:65:41:23
Network Name,sharednet1
Created At,2025-06-26T13:26:31Z
Keypair,ahmed_offsechq_com-jupyter
Reservation Id,None
Host Id,f873e03657a264d6bef7c4cf1f8a1d921944344e7f10ab418f462bbc


In [5]:
security_groups = [
  {'name': "allow-ssh", 'port': 22, 'description': "Enable SSH traffic on TCP port 22"},
  {'name': "allow-8888", 'port': 8888, 'description': "Enable TCP port 8888 (used by Jupyter)"},
  {'name': "allow-8000", 'port': 8000, 'description': "Enable TCP port 8000 (used by MLFlow)"},
  {'name': "allow-9000", 'port': 9000, 'description': "Enable TCP port 9000 (used by MinIO API)"},
  {'name': "allow-9001", 'port': 9001, 'description': "Enable TCP port 9001 (used by MinIO Web UI)"}
]

In [6]:
chi.clients.connection()

In [7]:
# configure openstacksdk for actions unsupported by python-chi
os_conn = chi.clients.connection()
nova_server = chi.nova().servers.get(s.id)

for sg in security_groups:

  if not os_conn.get_security_group(sg['name']):
      os_conn.create_security_group(sg['name'], sg['description'])
      os_conn.create_security_group_rule(sg['name'], port_range_min=sg['port'], port_range_max=sg['port'], protocol='tcp', remote_ip_prefix='0.0.0.0/0')

  nova_server.add_security_group(sg['name'])

print(f"updated security groups: {[group.name for group in nova_server.list_security_group()]}")

updated security groups: ['allow-8000', 'allow-8888', 'allow-9000', 'allow-9001', 'allow-ssh', 'default']


In [8]:
s.refresh()
s.check_connectivity()

Checking connectivity to 129.114.27.235 port 22.


Connection successful


In [9]:
s.execute("git clone https://github.com/A7med7x7/ReproGen.git")

/opt/conda/lib/python3.10/site-packages/paramiko/client.py:889: UserWarning: Unknown ssh-ed25519 host key for 129.114.27.235: b'8fa7f777922d05dfa90c70268b0b31d5'
  warnings.warn(
Cloning into 'ReproGen'...


<Result cmd='git clone https://github.com/A7med7x7/ReproGen.git' exited=0>

In [10]:
dirs = str(s.execute("ls"))
if "ReproGen" in dirs:
    print("Successfully clone the repo")
else:
    print("None")

ReproGen
cc_my_mounting_point.failure
openrc
Successfully clone the repo


In [11]:
s.execute("chmod +x ReproGen/scripts/setup.sh")

<Result cmd='chmod +x ReproGen/scripts/setup.sh' exited=0>

In [12]:
s.associate_floating_ip()

ResourceError: None of the ports can route to floating ip 129.114.27.242 on server 409303e0-830a-4204-b5fb-615f79cfe562

## Open an SSH session

Finally, open an SSH sesson on your server. From your local terminal, run

    ssh -i ~/.ssh/id_rsa_chameleon cc@A.B.C.D

where

-   in place of `~/.ssh/id_rsa_chameleon`, substitute the path to your own key that you had uploaded to KVM@TACC
-   in place of `A.B.C.D`, use the floating IP address you just associated to your instance.